In [2]:
# Create a connection

from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)

In [3]:
# Create a keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS killrvideo
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

In [4]:
# Set the keyspace
try:
    session.set_keyspace('killrvideo')
except Exception as e:
    print(e)

In [5]:
# partition key = tag and video_id = clustering column
query = """
CREATE TABLE IF NOT EXISTS videos_by_tag(
    tag text,
    video_id text,
    added_date text,
    title text,
    PRIMARY KEY ((tag),video_id)
)
"""
try:
    session.execute(query)
except Exception as e:
    print(e)

- Insert data from videos-by-tag.csv into the videos_by_tag table

In [6]:
import csv

In [8]:
file = r'C:\Users\abdul\cassandra-fundamentals\foundations-of-apache-cassandra\videos-by-tag.csv'

with open(file) as f:
    reader = csv.reader(f)
    next(reader)
    for line in reader:
        query = "INSERT INTO videos_by_tag(tag, video_id, added_date, title)"
        query = query + "VALUES(%s, %s, %s, %s)"
        session.execute(query, (line[0], line[1], line[2], line[3]))   

- Lets confirm that the data has been correctly inserted

In [9]:
query = "SELECT * FROM videos_by_tag"
rows = session.execute(query)
for row in rows:
    print(row.tag, row.video_id, row.added_date, row.title)

datastax 4845ed97-14bd-11e5-8a40-8338255b7e33 2013-10-16 DataStax Studio
datastax 5645f8bd-14bd-11e5-af1a-8638355b8e3a 2013-04-16 What is DataStax Enterprise?
cassandra 1645ea59-14bd-11e5-a993-8138354b7e31 2014-01-29 Cassandra History
cassandra 245e8024-14bd-11e5-9743-8238356b7e32 2012-04-03 Cassandra & SSDs
cassandra 3452f7de-14bd-11e5-855e-8738355b7e3a 2013-03-17 Cassandra Intro


- Lets us query data with tag = datastax

In [11]:
query = "SELECT * FROM videos_by_tag WHERE tag='datastax'"
rows = session.execute(query)
for row in rows:
    print(row.tag, row.video_id, row.added_date, row.title)

datastax 4845ed97-14bd-11e5-8a40-8338255b7e33 2013-10-16 DataStax Studio
datastax 5645f8bd-14bd-11e5-af1a-8638355b8e3a 2013-04-16 What is DataStax Enterprise?


- Lets us query data with tag = cassandra

In [12]:
query = "SELECT * FROM videos_by_tag WHERE tag='cassandra'"
rows = session.execute(query)
for row in rows:
    print(row.tag, row.video_id, row.added_date, row.title)

cassandra 1645ea59-14bd-11e5-a993-8138354b7e31 2014-01-29 Cassandra History
cassandra 245e8024-14bd-11e5-9743-8238356b7e32 2012-04-03 Cassandra & SSDs
cassandra 3452f7de-14bd-11e5-855e-8738355b7e3a 2013-03-17 Cassandra Intro


In [13]:
query = "DROP TABLE IF EXISTS videos_by_tag"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [14]:
session.shutdown()
cluster.shutdown()